1) Obtain subset of trnl sequences (e.g. chenopods) using sqlite, fasta_to_dict; then filter for length?

2) Add primer back on to ends (for better alignment)--see consensus scripts

3) Align with muscle

4) Check alignment in Geneious, delete uneven beginning (if necessary), output fixed alignment

5) Import into new notebook, specific to synapomizer (maybe write functions to module first, then import)

## Get all genera of subgroup (family, subfamily, etc.)

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from __future__ import division
from Bio import AlignIO
import sys
sys.path.append('/Users/hughcross/analysis/git_projects/consensus_builder/')
from sequence_funcs import *
from file_funcs import make_list


In [2]:
sqlite_file = '/Users/hughcross/analysis/camel2016/Camel_Plant_Reference_db.sqlite'

In [3]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [4]:
df = pd.read_sql('select * from plant_APNI_taxonomy', conn)
df.head()

,Taxon,TaxonCode,Tax_ID,regnum,division,class,subclass,superorder,order,family,subfamily,tribe,genus,species,subspecies,varietas
0,Anthocercis intricata,Anthocercis_intricata_195720,195720,Plantae,Charophyta,Equisetopsida,Magnoliidae,Asteranae,Solanales,Solanaceae,Nicotianoideae,Anthocercideae,Anthocercis,intricata,None,None
1,Myoporum platycarpum,Myoporum_platycarpum_796003,796003,Plantae,Charophyta,Equisetopsida,Magnoliidae,Asteranae,Lamiales,Scrophulariaceae,None,Myoporeae,Myoporum,platycarpum,None,None
2,Saphesia flaccida,Saphesia_flaccida_216014,216014,Plantae,Charophyta,Equisetopsida,Magnoliidae,Caryophyllanae,Caryophyllales,Aizoaceae,None,None,Saphesia,flaccida,None,None
3,Spyridium ulicinum,Spyridium_ulicinum_323969,323969,Plantae,Charophyta,Equisetopsida,Magnoliidae,Rosanae,Rosales,Rhamnaceae,None,Pomaderreae,Spyridium,ulicinum,None,None
4,Austrostipa wakoolica,Austrostipa_wakoolica_1030042,1030042,Plantae,Charophyta,Equisetopsida,Magnoliidae,Lilianae,Poales,Poaceae,Pooideae,Stipeae,Austrostipa,wakoolica,None,None


In [5]:
# get all chenopod taxa
chenos = df.ix[df.family == 'Chenopodiaceae']
len(chenos)

509

In [6]:
# get list of all cheno genera
cheno_gen = chenos['genus'].tolist()
chengen = set(cheno_gen)
print(len(cheno_gen))
print(len(chengen))

509
50


In [7]:
# now get trnl as dict
trnl_seqs = fasta_to_dict('/Users/hughcross/analysis/camel2016/trnl_refs/trim_derep_trnl_ref_seqs_9nov16.fasta')
len(trnl_seqs)

3324

In [8]:
cheno_trnl = {}
for k,v in trnl_seqs.iteritems():
    genus = k.split('_')[0]
    if genus in chengen:
        cheno_trnl[k]=v
print(len(cheno_trnl))

64


In [9]:
cheno_trnl['Atriplex_nummularia:AD_b76;size=3;']

'GGACTTAATTGGCTTGAGCCTTAGTATGGAGACCTACTAAGTGAAAACTTTCAAATTCAGAGAAACCCTGGAATTAATAAAAATGGGCAATCCTGAGCCAACTCCTTTTGTCAAAAGCAAAAAAGAAAGATTCAGAAAGCAAGAATAAAAAAAG'

### Add primers to 3` end

In [10]:
# now add end primer on to ends 
## from script: build_consensus_w_usearch_clustering.py in build_consensus git project
### make into a function
def add_trnl_primer(seqdict, dummy):
    """adds primer sequence to beginning and/or end of trnl sequences for improved alignment"""
    new_seqdict = {}
    for k,v in seqdict.iteritems():
        seqid = k
        seq = v
        if dummy == 'f':
            newseq = 'CGAAATCGGTAGACGCTAC'+v
        elif dummy == 'r':
            newseq = v+'GATAGGTGCAGAGACTCAATGG'
        elif dummy == 'b':
            newseq = 'CGAAATCGGTAGACGCTAC'+v+'GATAGGTGCAGAGACTCAATGG'
        new_seqdict[seqid]=newseq
    return new_seqdict


In [11]:
primed_cheno_trnl = add_trnl_primer(cheno_trnl, 'r')

In [12]:
primed_cheno_trnl['Atriplex_nummularia:AD_b76;size=3;']

'GGACTTAATTGGCTTGAGCCTTAGTATGGAGACCTACTAAGTGAAAACTTTCAAATTCAGAGAAACCCTGGAATTAATAAAAATGGGCAATCCTGAGCCAACTCCTTTTGTCAAAAGCAAAAAAGAAAGATTCAGAAAGCAAGAATAAAAAAAGGATAGGTGCAGAGACTCAATGG'

### Align with Muscle

In [13]:
# output to file
ch_out = open('chenopod_trnl_derep_seqs.fasta', 'w')
for key,value in primed_cheno_trnl.iteritems():
    ch_out.write('>'+key+'\n'+value+'\n')
ch_out.close()

In [14]:
%%bash
muscle -in chenopod_trnl_derep_seqs.fasta -out chenopod_trnl_aln


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

chenopod_trnl_derep_seqs 64 seqs, max length 180, avg  length 154
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00      8 MB(0%)  Iter   1  100.00%  Align node
00:00:00      8 MB(0%)  Iter   1  100.00%  Root alignment
00:00:00      8 MB(0%)  Iter   2  100.00%  Refine tree
00:00:00      8 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00      8 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00      8 MB(0%)  Iter   3  100.00%  Refine biparts
00:00:00      8 MB(0%)  Iter   4  100.00%  Refine biparts
00:00:00      8 MB(0%)  Iter   5  100.00%  Refine biparts
00:00:00      8 MB(0%)  Iter   5  100.00%  Refine biparts


### alignment still messy at end, though better with MAFFT re-alignment and editing. Try another family

In [15]:
# get all chenopod taxa
taters = df.ix[df.family == 'Solanaceae']
len(taters)

569

In [16]:
# get list of all cheno genera
tat_gen = taters['genus'].tolist()
tatgen = set(tat_gen)
print(len(tat_gen))
print(len(tatgen))

569
12


In [17]:
tat_trnl = {}
for k,v in trnl_seqs.iteritems():
    genus = k.split('_')[0]
    if genus in tatgen:
        tat_trnl[k]=v
print(len(tat_trnl))

95


In [18]:
primed_tat_trnl = add_trnl_primer(tat_trnl, 'r')

In [19]:
# output to file
ch_out = open('potato_trnl_derep_seqs.fasta', 'w')
for key,value in primed_tat_trnl.iteritems():
    ch_out.write('>'+key+'\n'+value+'\n')
ch_out.close()

In [20]:
%%bash
muscle -in potato_trnl_derep_seqs.fasta -out tat_trnl_aln


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

potato_trnl_derep_seqs 95 seqs, max length 164, avg  length 153
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00     11 MB(0%)  Iter   1  100.00%  Align node
00:00:00     11 MB(0%)  Iter   1  100.00%  Root alignment
00:00:00     12 MB(0%)  Iter   2  100.00%  Refine tree
00:00:00     12 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00     12 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00     13 MB(0%)  Iter   3  100.00%  Refine biparts
00:00:01     13 MB(0%)  Iter   4  100.00%  Refine biparts
00:00:01     13 MB(0%)  Iter   5  100.00%  Refine biparts
00:00:01     13 MB(0%)  Iter   5  100.00%  Refine biparts


### try Asteraceae

In [21]:
# get all chenopod taxa
daisies = df.ix[df.family == 'Asteraceae']
len(daisies)

1594

In [22]:
# get list of all cheno genera
dai_gen = daisies['genus'].tolist()
daigen = set(dai_gen)
print(len(dai_gen))
print(len(daigen))

1594
134


In [23]:
## make another function from filtering dictionary
def filter_genus(seqdict, genera):
    """filter a sequence dictionary by the genus name (first name in id)"""
    filt_seqdict = {}
    for k,v in seqdict.iteritems():
        genus = k.split('_')[0]
        if genus in genera:
            filt_seqdict[k]=v
    return filt_seqdict

In [24]:
daisy_trnl = filter_genus(trnl_seqs, daigen)
print(len(daisy_trnl))

225


In [25]:
primed_daisy_trnl = add_trnl_primer(daisy_trnl, 'r')

In [26]:
# output to file
ch_out = open('daisy_trnl_derep_seqs.fasta', 'w')
for key,value in primed_daisy_trnl.iteritems():
    ch_out.write('>'+key+'\n'+value+'\n')
ch_out.close()

In [27]:
%%bash
muscle -in daisy_trnl_derep_seqs.fasta -out daisy_trnl_aln


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

daisy_trnl_derep_seqs 225 seqs, max length 173, avg  length 152
00:00:00      2 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00     24 MB(0%)  Iter   1  100.00%  Align node
00:00:00     24 MB(0%)  Iter   1  100.00%  Root alignment
00:00:00     27 MB(0%)  Iter   2  100.00%  Refine tree
00:00:00     27 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00     27 MB(0%)  Iter   2  100.00%  Root alignment
00:00:01     28 MB(0%)  Iter   3  100.00%  Refine biparts
00:00:03     29 MB(0%)  Iter   4  100.00%  Refine biparts
00:00:03     29 MB(0%)  Iter   5  100.00%  Refine biparts
00:00:03     29 MB(0%)  Iter   5  100.00%  Refine biparts


### Try grasses

In [28]:
# get all chenopod taxa
grasses = df.ix[df.family == 'Poaceae']
len(grasses)

1835

In [30]:
# get list of all cheno genera
poa_gen = grasses['genus'].tolist()
poagen = set(poa_gen)
print(len(poa_gen))
print(len(poagen))

1835
113


In [31]:
grass_trnl = filter_genus(trnl_seqs, poagen)
print(len(grass_trnl))

389


In [32]:
primed_grass_trnl = add_trnl_primer(grass_trnl, 'r')

In [33]:
# output to file
ch_out = open('grass_trnl_derep_seqs.fasta', 'w')
for key,value in primed_grass_trnl.iteritems():
    ch_out.write('>'+key+'\n'+value+'\n')
ch_out.close()

In [34]:
%%bash
muscle -in grass_trnl_derep_seqs.fasta -out grass_trnl_aln


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

grass_trnl_derep_seqs 389 seqs, max length 179, avg  length 156
00:00:00      3 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      3 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
00:00:01     43 MB(0%)  Iter   1  100.00%  Align node
00:00:01     43 MB(0%)  Iter   1  100.00%  Root alignment
00:00:01     47 MB(0%)  Iter   2  100.00%  Refine tree
00:00:01     48 MB(0%)  Iter   2  100.00%  Root alignment
00:00:01     48 MB(0%)  Iter   2  100.00%  Root alignment
00:00:06     50 MB(0%)  Iter   3  100.00%  Refine biparts
00:00:10     51 MB(0%)  Iter   4  100.00%  Refine biparts
00:00:10     51 MB(0%)  Iter   5  100.00%  Refine biparts
00:00:10     51 MB(0%)  Iter   5  100.00%  Refine biparts


### Try Casurinaceae, combining commands into fewer cells

In [35]:
# get all chenopod taxa
bloaks = df.ix[df.family == 'Casuarinaceae']
len(bloaks)
# get list of all genera
oak_gen = bloaks['genus'].tolist()
oakgen = set(oak_gen)
print(len(oak_gen))
print(len(oakgen))
bloak_trnl = filter_genus(trnl_seqs, oakgen)
print(len(bloak_trnl))
primed_bloak_trnl = add_trnl_primer(bloak_trnl, 'r')

26
3
3


In [36]:
# output to file
ch_out = open('black_oak_trnl_derep_seqs.fasta', 'w')
for key,value in primed_bloak_trnl.iteritems():
    ch_out.write('>'+key+'\n'+value+'\n')
ch_out.close()

### Malvaceae

In [37]:
# get all malvaceae taxa
malvs = df.ix[df.family == 'Malvaceae']
len(malvs)
# get list of all genera
mal_gen = malvs['genus'].tolist()
malgen = set(mal_gen)
print(len(mal_gen))
print(len(malgen))
malv_trnl = filter_genus(trnl_seqs, malgen)
print(len(malv_trnl))
primed_malv_trnl = add_trnl_primer(malv_trnl, 'r')

288
36
31


In [38]:
# output to file
ch_out = open('malvaceae_trnl_derep_seqs.fasta', 'w')
for key,value in primed_malv_trnl.iteritems():
    ch_out.write('>'+key+'\n'+value+'\n')
ch_out.close()

In [39]:
%%bash
muscle -in malvaceae_trnl_derep_seqs.fasta -out malv_trnl_aln


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

malvaceae_trnl_derep_seqs 31 seqs, max length 195, avg  length 159
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      1 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00      4 MB(0%)  Iter   1  100.00%  Align node
00:00:00      4 MB(0%)  Iter   1  100.00%  Root alignment
00:00:00      5 MB(0%)  Iter   2  100.00%  Refine tree
00:00:00      5 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00      5 MB(0%)  Iter   2  100.00%  Root alignment
00:00:00      5 MB(0%)  Iter   3  100.00%  Refine biparts
00:00:00      5 MB(0%)  Iter   4  100.00%  Refine biparts
00:00:00      5 MB(0%)  Iter   5  100.00%  Refine biparts
00:00:00      5 MB(0%)  Iter   5  100.00%  Refine biparts
